In [ ]:
!apt-get install -y portaudio19-dev libportaudio2
!pip install openai sounddevice scipy PyPDF2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (333 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ..

In [ ]:
import openai
import PyPDF2
import sounddevice as sd
from scipy.io.wavfile import write
import tempfile

# Initialize OpenAI API key
openai.api_key = "api-key"

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, "rb") as file:
        # Use PdfReader instead of the deprecated PdfFileReader
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)): # Use len(pdf_reader.pages) to get page count
            text += pdf_reader.pages[page_num].extract_text() # Access pages using pages attribute
    return text

def get_resume_score(resume_text, job_description):
    """Get a score for the resume against a job description."""
    prompt = f"""
    You are a recruitment AI. Evaluate this resume based on the job description provided.
    Return a score out of 100, with a detailed explanation if possible.

    Job Description:
    {job_description}

    Resume:
    {resume_text}

    Score (Formatted as x/100):
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract the score from the response
    score_output = response.choices[0].message.content
    return score_output

def transcribe_audio(audio_file_path):
    """Use OpenAI's Whisper ASR to transcribe audio."""
    with open(audio_file_path, "rb") as audio_file:
        transcript = openai.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1"
        )
    return transcript.text

def evaluate_interview_response(question, transcribed_response):
    """Evaluate a user's interview response based on a sample question and return a score out of 100."""
    prompt = f"""
    You are an interview coach. Rate the user's response to the interview question out of 100,
    considering relevance, clarity, and alignment with the job description. Provide feedback as well.

    Interview Question:
    {question}

    User's Response:
    {transcribed_response}

    Score and Feedback (Formatted as x/100):
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract the score and feedback from the response
    score_output = response.choices[0].message.content
    return score_output

# Main function to load PDF, get resume score, and evaluate interview responses
def main(pdf_path, job_description, interview_questions, audio_file_paths):
    resume_text = extract_text_from_pdf(pdf_path)
    resume_score = get_resume_score(resume_text, job_description)
    print("Resume Score and Feedback:")
    print(resume_score)

    print("\nInterview Responses Evaluation:")
    for question, audio_file in zip(interview_questions, audio_file_paths):
        print(f"\nQuestion: '{question}'")

        transcribed_response = transcribe_audio(audio_file)  # Transcribe the corresponding audio file
        print(f"Transcribed Response: {transcribed_response}")

        response_score = evaluate_interview_response(question, transcribed_response)
        print(f"Score and Feedback for the response to '{question}':")
        print(response_score)

# Usage
job_description = """
Job Description

Visa U.S.A. Inc., a Visa Inc. company, needs a Data Scientist (multiple openings) in Washington DC to

Brainstorm innovative ways to use unique data to answer business problems.
Communicate with clients to understand the challenges they face and convince them with data.
Extract and understand data to form an opinion on how to best help clients and derive relevant insights.
Develop visualizations to make complex analyses accessible to a broad audience.
Find opportunities to craft products out of analyses that are suitable for multiple clients.
Work with stakeholders throughout the organization to identify opportunities for leveraging Visa data to drive business solutions.
Mine and analyze data from company databases to drive optimization and improvement of product, marketing techniques and business strategies for Visa and its clients.
Assess the effectiveness and accuracy of new data sources and data gathering techniques.
Develop custom data models and algorithms to apply to data sets.
Use predictive modeling to increase and optimize customer experiences, revenue generation, data insights, advertising targeting and other business outcomes.
Develop processes and tools to monitor and analyze model performance and data accuracy.
Domestic travel may be required up to 5% of the time.
Position reports to the Washington, DC office and may allow for partial telecommuting.

Qualifications

Basic Qualifications:

Employer will accept a Bachelor's degree in Statistics, Operations Research, Economics, Computer Science or related field and 2 years of experience in the job offered or in a data scientist-related occupation.
Position requires experience in the following:
Extracting and aggregating data from large data sets using SQL or other similar tool;
Developing visualizations in Tableau or other similar software, and communicating insights from complex analyses to a broader audience;
Understanding and analyzing data using statistical software, such as Python, SAS, R, or Stata;
Data mining and statistical modeling (e.g., regression modeling, clustering techniques, or decision trees, etc.); and
Developing custom data models and algorithms to apply to data sets.
"""
pdf_path = "/content/Weslee_Hwang_Resume.pdf"
interview_questions = [
    "Tell me about yourself.",

]

audio_file_paths = [
    "/content/interview_question1.m4a",  # Audio file for question 1

]
"""
"Why are you interested in this role?",
    "Describe a challenging project you've worked on and how you managed it."
"""
"""
"path/to/your_audio_file_2.wav",  # Audio file for question 2
"path/to/your_audio_file_3.wav",  # Audio file for question 3
"""

main(pdf_path, job_description, interview_questions, audio_file_paths)

Resume Score and Feedback:
90/100

Explanation:

This candidate is in the strong batch for this role and there are several reasons for this assessment: 

1. Work Experience: The candidate has worked as an Associate Data Scientist/Statistician, where he conducted statistical analysis on large data sets using languages (Python, SQL) required by the job description.

2. Hands-on Projects: The candidate has worked on various real-world projects involving neural networks, ETL pipelines, and data visualization which directly mirrors the responsibilities outlined in the job description.

3. Technical Skills: The candidate is proficient in Python, SQL, and R along with ML Packages, skills which are expressly required in the job description. Moreover, he possesses experience with Tableau-like software for data visualizations (matplotlib, ggplot2) and ETL pipeline, which is a need for this role too.

4. Education: While the candidate's Bachelor's degree is in Computational Data Science (which ne